In [2]:
# Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)

US Configuration

In [3]:
census_regions = [
    {
        "number" : 0,
        "name" : "United States",
        "states" : ["United States"]
    },
    {
        "number" : 1,
        "name" : "Northeast",
        "states" :["Maine", "New Hampshire", "Vermont", "Massachusetts", "Connecticut", "Rhode Island", "New Jersey", "New York", "Pennsylvania"]
    },
    {
        "number" : 3,
        "name" : "South",
        "states" : ["Maryland", "Delaware", "West Virginia", "Virginia", "Kentucky", "Tennessee", "North Carolina", "South Carolina", "Georgia", "Florida", "Alabama", "Mississippi", "Arkansas", "Louisiana", "Oklahoma", "Texas", "District of Columbia", "Puerto Rico"]
    },
    {
        "number" : 2,
        "name" : "Midwest",
        "states" : ["North Dakota", "South Dakota", "Nebraska", "Kansas", "Missouri", "Iowa", "Minnesota", "Wisconsin", "Illinois", "Michigan", "Indiana", "Ohio"]
    },
    {
        "number" : 4,
        "name" : "West",
        "states" : ["Washington", "Idaho", "Montana", "Wyoming", "Oregon", "California", "Nevada", "Utah", "Colorado", "Arizona", "New Mexico", "Alaska", "Hawaii"]
    }
]
print("Census Regions")
for census_region in census_regions:
    pp.pprint(census_region)

Census Regions
{'name': 'United States', 'number': 0, 'states': ['United States']}
{   'name': 'Northeast',
    'number': 1,
    'states': [   'Maine',
                  'New Hampshire',
                  'Vermont',
                  'Massachusetts',
                  'Connecticut',
                  'Rhode Island',
                  'New Jersey',
                  'New York',
                  'Pennsylvania']}
{   'name': 'South',
    'number': 3,
    'states': [   'Maryland',
                  'Delaware',
                  'West Virginia',
                  'Virginia',
                  'Kentucky',
                  'Tennessee',
                  'North Carolina',
                  'South Carolina',
                  'Georgia',
                  'Florida',
                  'Alabama',
                  'Mississippi',
                  'Arkansas',
                  'Louisiana',
                  'Oklahoma',
                  'Texas',
                  'District of Columbia',
         

In [4]:
def fixRegion(code):
    region_name = ""
    for region in census_regions:
        if region["number"] == code:
            region_name = region["name"]
            break
    if region_name == "":
        region_name = "Other"
        print(str(code) + " not found")
    return region_name

# CDC Standard age ranges 0-17, 18-29, 30-49, and 50-64
# CDC COVID Reporting Age Ranges https://www.cdc.gov/nchs/nvss/vsrr/covid_weekly/index.htm
def getAgeRange(age):
    age_range = ""
    if age == 0:
        age_range = "< 1"
    elif age == 999:
        age_range = "Total"
    elif age < 5:
        age_range = "1-4"
    elif age < 15:
        age_range = "5-14"
    elif age < 25:
        age_range = "15-24"
    elif age < 35:
        age_range = "25-34"
    elif age < 45:
        age_range = "35-44"
    elif age < 55:
        age_range = "45-54"
    elif age < 65:
        age_range = "55-64"
    elif age < 75:
        age_range = "65-74"
    elif age < 85:
        age_range = "75-84"
    elif age == 85:
        age_range = "85+"
    return age_range

def fixSex(code):
    sex = ""
    if code == 0:
        sex = "Population 2019"
    elif code == 1:
        sex = "Male"
    elif code == 2:
        sex = "Female"
    else:
        print(str(code) + " is not a sex")
    return sex

def us_date(x):
    month = x[5:7]
    day = x[8:11]
    year = x[0:4]
    conversion = month + "/" + day +"/"+ year
    return conversion

def removeDecimal(data):
    strData = str(data)
    decimalLocation = strData.find(".")
    if decimalLocation > -1:
        return strData[0:decimalLocation]
    else:
        return strData

def emptyNan(value):
    if (value == "nan"):
        return ""
    else:
        return value

def printColumns(df, label):
    print(label)
    print(df.columns)

Download US Data

In [4]:
us_states_census_demographics = "https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/asrh/sc-est2019-agesex-civ.csv"
us_states_census_demographics_request = requests.get(us_states_census_demographics).content
us_demographics = pd.read_csv(io.StringIO(us_states_census_demographics_request.decode('utf-8')))
currentTime = datetime.now()
us_demographics["Downloaded"] = currentTime
us_demographics["Country"] = "United States"
us_demographics["REGION"] = us_demographics["REGION"].apply(lambda x: fixRegion(x))
us_demographics["SEX"] = us_demographics["SEX"].apply(lambda x: fixSex(x))
print(us_demographics["AGE"].sort_values(ascending = True).unique())
us_demographics["Age Range"] = us_demographics["AGE"].apply(lambda x: getAgeRange(x))
keep_columns = ["REGION","STATE","NAME","SEX","AGE","POPEST2019_CIV","Downloaded","Country", "Age Range"]
us_demographics = us_demographics[keep_columns]
us_demographics.rename(columns = {'REGION': 'Census Region',
                                  'NAME' : 'State Name',
                                  'STATE' : 'FIPS',
                                  'POPEST2019_CIV' : 'Population 2019',
                                  'SEX' : 'Sex',
                                  'AGE' : 'Age'}, 
                       inplace = True)

us_demographics.head()

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85 999]


,Census Region,FIPS,State Name,Sex,Age,Population 2019,Downloaded,Country,Age Range
0,United States,0,United States,Population 2019,0,3783052,2020-10-14 00:39:06.662531,United States,< 1
1,United States,0,United States,Population 2019,1,3829599,2020-10-14 00:39:06.662531,United States,1-4
2,United States,0,United States,Population 2019,2,3922044,2020-10-14 00:39:06.662531,United States,1-4
3,United States,0,United States,Population 2019,3,3998665,2020-10-14 00:39:06.662531,United States,1-4
4,United States,0,United States,Population 2019,4,4043323,2020-10-14 00:39:06.662531,United States,1-4


In [5]:
us_sex = us_demographics.drop(columns=["Age Range"]).loc[us_demographics["Age"]==999].copy()
us_sex = us_sex.pivot_table(
    index=["Downloaded","Country","Census Region","State Name","FIPS","Sex"],
    columns='Age',
    values = 'Population 2019',
    aggfunc='first'
).reset_index().rename_axis(None, axis=1)
us_sex["Total Population"] = us_sex[999]
us_sex = us_sex.sort_values(["FIPS", "Sex"])
us_sex = us_sex.drop(columns=[999])
us_sex = us_sex.pivot_table(
    index=["Downloaded","Country","Census Region","State Name","FIPS"],
    columns='Sex',
    values = 'Total Population',
    aggfunc='first'
).reset_index().rename_axis(None, axis=1)
print(us_sex.columns)
us_sex["Pct Male"] = us_sex["Male"]/us_sex["Population 2019"]
us_sex["Pct Female"] = us_sex["Female"]/us_sex["Population 2019"]
us_sex = us_sex.sort_values(["FIPS"])
us_sex.head()

Index(['Downloaded', 'Country', 'Census Region', 'State Name', 'FIPS',
       'Female', 'Male', 'Population 2019'],
      dtype='object')


,Downloaded,Country,Census Region,State Name,FIPS,Female,Male,Population 2019,Pct Male,Pct Female
38,2020-10-14 00:39:06.662531,United States,United States,United States,0,166382969,160669633,327052602,0.491265,0.508735
21,2020-10-14 00:39:06.662531,United States,South,Alabama,1,2531653,2357694,4889347,0.482210,0.517790
39,2020-10-14 00:39:06.662531,United States,West,Alaska,2,347065,365049,712114,0.512627,0.487373
40,2020-10-14 00:39:06.662531,United States,West,Arizona,4,3658425,3600665,7259090,0.496022,0.503978
22,2020-10-14 00:39:06.662531,United States,South,Arkansas,5,1535409,1477133,3012542,0.490328,0.509672


In [6]:
us_age = us_demographics[["Census Region","FIPS","State Name","Age", "Age Range", "Population 2019"]].copy()
us_age = us_age.pivot_table(index=["Census Region","FIPS","State Name"], 
                      columns='Age', 
                      values='Population 2019', 
                      aggfunc='first').reset_index().rename_axis(None, axis=1)
us_age["Total Population"] = us_age[999]
us_age["< 1"] = us_age[0]
us_age["1-4"] = us_age[1]+us_age[2]+us_age[3]+us_age[4]
us_age["5-14"] = us_age[5]+us_age[6]+us_age[7]+us_age[8]+us_age[9]+us_age[10]+us_age[11]+us_age[12]+us_age[13]+us_age[14]
us_age["15-24"] = us_age[15]+us_age[16]+us_age[17]+us_age[18]+us_age[19]+us_age[20]+us_age[21]+us_age[22]+us_age[23]+us_age[24]
us_age["25-34"] = us_age[25]+us_age[26]+us_age[27]+us_age[28]+us_age[29]+us_age[30]+us_age[31]+us_age[32]+us_age[33]+us_age[34]
us_age["35-44"] = us_age[35]+us_age[36]+us_age[37]+us_age[38]+us_age[39]+us_age[40]+us_age[41]+us_age[42]+us_age[43]+us_age[44]
us_age["45-54"] = us_age[45]+us_age[46]+us_age[47]+us_age[48]+us_age[49]+us_age[50]+us_age[51]+us_age[52]+us_age[53]+us_age[54]
us_age["55-64"] = us_age[55]+us_age[56]+us_age[57]+us_age[58]+us_age[59]+us_age[60]+us_age[61]+us_age[62]+us_age[63]+us_age[64]
us_age["65-74"] = us_age[65]+us_age[66]+us_age[67]+us_age[68]+us_age[69]+us_age[70]+us_age[71]+us_age[72]+us_age[73]+us_age[74]
us_age["75-84"] = us_age[75]+us_age[76]+us_age[77]+us_age[78]+us_age[79]+us_age[80]+us_age[81]+us_age[82]+us_age[83]+us_age[84]
us_age["85+"] = us_age[85]
us_age["Pct < 1"] = us_age["< 1"]/us_age["Total Population"]
us_age["Pct 1-4"] = us_age["1-4"]/us_age["Total Population"]
us_age["Pct 5-14"] = us_age["5-14"]/us_age["Total Population"]
us_age["Pct 15-24"] = us_age["15-24"]/us_age["Total Population"]
us_age["Pct 25-34"] = us_age["25-34"]/us_age["Total Population"]
us_age["Pct 35-44"] = us_age["35-44"]/us_age["Total Population"]
us_age["Pct 45-54"] = us_age["45-54"]/us_age["Total Population"]
us_age["Pct 55-64"] = us_age["55-64"]/us_age["Total Population"]
us_age["Pct 65-74"] = us_age["65-74"]/us_age["Total Population"]
us_age["Pct 75-84"] = us_age["75-84"]/us_age["Total Population"]
us_age["Pct 85+"] = us_age["85+"]/us_age["Total Population"]
us_age = us_age.drop(columns=["Census Region","State Name",0,999])
age_order = [
    'FIPS', 'Total Population',
    '< 1', 1, 2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
    40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
    70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, '85+',
    '1-4', '5-14', '15-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84',
    'Pct < 1', 'Pct 1-4', 'Pct 5-14', 'Pct 15-24', 'Pct 25-34', 'Pct 35-44', 'Pct 45-54', 'Pct 55-64', 'Pct 65-74', 'Pct 75-84', 'Pct 85+'
]
us_age = us_age[age_order]
us_age = us_age.sort_values(["FIPS"])
us_age.head()

,FIPS,Total Population,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
38,0,327052602,3783052,3829599,3922044,3998665,4043323,4028281,4017227,4022319,4066194,4061874,4060940,4189261,4208387,4175221,4164459,4175459,4150420,4141202,4222496,4272909,4203852,4207331,4225571,4271168,4379797,4478029,4555029,4680384,4770188,4761228,4572098,4461418,4382364,4396137,4425400,4282329,4340779,4330751,4278041,4358571,4084430,4002096,3958280,3840006,3954899,3827009,3879028,4049147,4275506,4324119,4091962,4000602,3998899,4066500,4303847,4373250,4360049,4341663,4385025,4413491,4252458,4215037,4156618,3996074,3950547,3774597,3618069,3464437,3345475,3252423,3136704,3083083,3191048,2334433,2283164,2198286,2222392,1911261,1720817,1599909,1475278,1381641,1241341,1151190,1067757,6604958,15793631,40994163,42250205,45482275,41430182,40816619,42444212,31483433,15969872,0.011567,0.048291,0.125344,0.129185,0.139067,0.126677,0.124801,0.129778,0.096264,0.048830,0.020195
21,1,4889347,56901,58290,59073,59799,60294,59568,58599,59537,60023,60241,60897,63083,62906,61883,61729,61740,61799,61924,62938,64125,63587,64201,63943,63719,63922,65079,65208,67027,69478,68758,64852,61469,59980,59615,60721,58941,59921,60346,60696,62200,58159,57993,57852,55498,58174,57008,58838,61959,65460,64750,60738,59494,59786,61321,65925,66906,66695,67073,67308,68221,65605,65211,65365,63117,62042,59584,56766,54694,52697,51707,50567,49884,51612,37091,36845,35441,36173,30575,27572,26053,23977,22580,19594,18222,16660,91543,237456,608466,631898,642187,589780,615279,657543,501447,256847,0.011638,0.048566,0.124447,0.129240,0.131344,0.120626,0.125841,0.134485,0.102559,0.052532,0.018723
39,2,712114,9978,10012,10186,10509,10395,10414,10303,10286,10436,10157,9976,10016,9887,9509,9678,9488,9410,9343,8518,7525,8088,8617,9132,9252,9900,10318,10693,11456,11576,11552,10946,10809,10460,10822,10799,10303,10452,9962,9667,9685,8865,8589,8533,7954,8295,7827,7962,8230,8696,9086,8428,8197,8330,8595,9128,9426,9493,9636,9731,9894,9373,9168,9161,8614,8800,8210,7734,7220,6655,6442,5978,5621,5482,4013,3945,3665,3459,2950,2677,2327,1971,1784,1586,1411,1277,7181,41102,100662,89273,109431,92305,84479,93296,61300,23107,0.014012,0.057718,0.141357,0.125363,0.153671,0.129621,0.118631,0.131013,0.086082,0.032448,0.010084
40,4,7259090,81929,83065,85726,88192,90876,90858,90405,90319,91313,90797,92572,96605,96951,95849,95076,94742,91832,93119,97724,99334,95779,96720,97447,99492,100066,101835,102053,105016,106612,105885,99473,96012,93463,94419,94495,91292,93193,93126,92593,94337,87480,85369,85480,83038,86427,83705,82925,85342,90167,90646,85428,82772,81434,81904,87011,88220,88617,88864,90069,90909,88041,87973,87664,85106,85274,83374,81125,78896,77728,77376,76474,76787,81299,59741,58899,57521,58174,50088,45100,41784,38116,35128,31252,28331,25703,145737,347859,930745,966255,999263,892335,851334,880737,751699,411197,0.011286,0.047920,0.128218,0.133110,0.137657,0.122927,0.117278,0.121329,0.103553,0.056646,0.020076
22,5,3012542,36355,37006,37572,38610,38921,38404,37924,38827,38633,38959,38941,40404,41015,40146,39960,39598,39485,39395,38933,39714,40206,40211,40323,39367,38992,39539,39518,40912,42271,41927,39361,38289,37446,37354,37897,37550,38010,38198,38328,39332,36427,36037,35410,34319,35486,34449,34938,36485,37966,38136,35481,34716,34766,35572,38680,39707,39698,39360,39525,39697,38668,38092,37865,36917,36430,35478,34176,32682,31736,30888,30427,30017,31554,22864,23007,22169,22217,19362,17669,16670,14936,13764,12330,11253,10771,59912,152109,393213,396224,394514,369097,361189,385959,302829,161141,0.012068,0.050492,0.130525,0.131525,0.130957,0.122520,0.119895,0.128117,0.100523,0.053490

In [7]:
state_codes = pd.read_excel(r'C:\Users\janin\Downloads\US State Codes.xlsx')
state_codes = state_codes.drop(columns=["State Name"])
state_codes.head()

,FIPS,State Abbreviation,Status
0,0,US,0
1,1,AL,0
2,2,AK,0
3,4,AZ,0
4,5,AR,0


In [8]:
us_state_demographics = pd.merge(us_sex, us_age, how="left", on="FIPS")
us_state_demographics = pd.merge(us_state_demographics, state_codes, how="left", on="FIPS")
us_state_demographics = us_state_demographics.drop(columns=["FIPS", "Status"])
demographics_order = [
    'State Abbreviation', 'Census Region',
    'Population 2019', 'Female', 'Male', 'Pct Male', 'Pct Female',
    '< 1', 1, 2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
    40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
    70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, '85+',
    '1-4', '5-14', '15-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84',
    'Pct < 1', 'Pct 1-4', 'Pct 5-14', 'Pct 15-24', 'Pct 25-34', 'Pct 35-44', 'Pct 45-54', 'Pct 55-64', 'Pct 65-74', 'Pct 75-84', 'Pct 85+'
]
us_state_demographics = us_state_demographics[demographics_order]
us_state_demographics = us_state_demographics.sort_values(["State Abbreviation"])
us_state_demographics.head()

,State Abbreviation,Census Region,Population 2019,Female,Male,Pct Male,Pct Female,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
2,AK,West,712114,347065,365049,0.512627,0.487373,9978,10012,10186,10509,10395,10414,10303,10286,10436,10157,9976,10016,9887,9509,9678,9488,9410,9343,8518,7525,8088,8617,9132,9252,9900,10318,10693,11456,11576,11552,10946,10809,10460,10822,10799,10303,10452,9962,9667,9685,8865,8589,8533,7954,8295,7827,7962,8230,8696,9086,8428,8197,8330,8595,9128,9426,9493,9636,9731,9894,9373,9168,9161,8614,8800,8210,7734,7220,6655,6442,5978,5621,5482,4013,3945,3665,3459,2950,2677,2327,1971,1784,1586,1411,1277,7181,41102,100662,89273,109431,92305,84479,93296,61300,23107,0.014012,0.057718,0.141357,0.125363,0.153671,0.129621,0.118631,0.131013,0.086082,0.032448,0.010084
1,AL,South,4889347,2531653,2357694,0.482210,0.517790,56901,58290,59073,59799,60294,59568,58599,59537,60023,60241,60897,63083,62906,61883,61729,61740,61799,61924,62938,64125,63587,64201,63943,63719,63922,65079,65208,67027,69478,68758,64852,61469,59980,59615,60721,58941,59921,60346,60696,62200,58159,57993,57852,55498,58174,57008,58838,61959,65460,64750,60738,59494,59786,61321,65925,66906,66695,67073,67308,68221,65605,65211,65365,63117,62042,59584,56766,54694,52697,51707,50567,49884,51612,37091,36845,35441,36173,30575,27572,26053,23977,22580,19594,18222,16660,91543,237456,608466,631898,642187,589780,615279,657543,501447,256847,0.011638,0.048566,0.124447,0.129240,0.131344,0.120626,0.125841,0.134485,0.102559,0.052532,0.018723
4,AR,South,3012542,1535409,1477133,0.490328,0.509672,36355,37006,37572,38610,38921,38404,37924,38827,38633,38959,38941,40404,41015,40146,39960,39598,39485,39395,38933,39714,40206,40211,40323,39367,38992,39539,39518,40912,42271,41927,39361,38289,37446,37354,37897,37550,38010,38198,38328,39332,36427,36037,35410,34319,35486,34449,34938,36485,37966,38136,35481,34716,34766,35572,38680,39707,39698,39360,39525,39697,38668,38092,37865,36917,36430,35478,34176,32682,31736,30888,30427,30017,31554,22864,23007,22169,22217,19362,17669,16670,14936,13764,12330,11253,10771,59912,152109,393213,396224,394514,369097,361189,385959,302829,161141,0.012068,0.050492,0.130525,0.131525,0.130957,0.122520,0.119895,0.128117,0.100523,0.053490,0.019888
3,AZ,West,7259090,3658425,3600665,0.496022,0.503978,81929,83065,85726,88192,90876,90858,90405,90319,91313,90797,92572,96605,96951,95849,95076,94742,91832,93119,97724,99334,95779,96720,97447,99492,100066,101835,102053,105016,106612,105885,99473,96012,93463,94419,94495,91292,93193,93126,92593,94337,87480,85369,85480,83038,86427,83705,82925,85342,90167,90646,85428,82772,81434,81904,87011,88220,88617,88864,90069,90909,88041,87973,87664,85106,85274,83374,81125,78896,77728,77376,76474,76787,81299,59741,58899,57521,58174,50088,45100,41784,38116,35128,31252,28331,25703,145737,347859,930745,966255,999263,892335,851334,880737,751699,411197,0.011286,0.047920,0.128218,0.133110,0.137657,0.122927,0.117278,0.121329,0.103553,0.056646,0.020076
5,CA,West,39356141,19843586,19512555,0.495794,0.504206,462589,462713,477322,485894,495198,493458,494221,493396,504330,493445,492283,511109,512662,507455,505628,503712,501846,497188,515261,501692,493088,497749,512251,533604,557011,576604,588951,613288,640318,640758,611094,595453,577581,574306,575253,556953,559625,555569,542576,549416,510476,499294,496070,486794,500853,483161,483781,493411,516222,535475,504901,490446,479620,481731,506585,509027,503106,495480,495887,505031,475913,467624,457973,440326,435989,411959,392969,371355,358803,354786,334996,325934,330822,255411,249635,233568,229072,200181,182011,172849,158877,148337,135255,124947,116502,749846,1

In [9]:
state_codes = pd.read_excel(r'C:\Users\janin\Downloads\US State Codes.xlsx')
state_codes.head()

,State Name,FIPS,State Abbreviation,Status
0,United States,0,US,0
1,Alabama,1,AL,0
2,Alaska,2,AK,0
3,Arizona,4,AZ,0
4,Arkansas,5,AR,0


In [11]:
us_states_url = "https://covidtracking.com/data/download/all-states-history.csv"
us_states_request = requests.get(us_states_url).content
states=pd.read_csv(io.StringIO(us_states_request.decode('utf-8')))
currentTime = datetime.now()
states["Downloaded"] = currentTime
states["Country"] = "United States"
printColumns(states, "Pre Rename Columns")
states = states.drop(
    columns = [
        'deathConfirmed', 'deathProbable',
        'hospitalized',
        'negativeTestsAntibody', 'negativeTestsPeopleAntibody', 'negativeTestsViral',
        'pending',
        'positiveScore', 'positiveTestsAntibody', 'positiveTestsAntigen',
        'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
        'positiveTestsViral', 'positiveCasesViral',
        'totalTestEncountersViral', 'totalTestEncountersViralIncrease',
        'totalTestsAntibody', 'totalTestsAntigen',
        'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
        'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
        'totalTestsViral', 'totalTestsViralIncrease'
    ])
states.rename(
    columns = {
        'date': 'Time', 'state' : 'State Abbreviation', 'dataQualityGrade': 'Data Quality',
        'totalTestResults' : 'Total Tests', 'totalTestResultsIncrease' : 'Tests Daily',
        'negative' : 'Total Negative', 'negativeIncrease' : 'Negative Daily',
        'positive' : 'Total Positive', 'positiveIncrease' : 'Positive Daily',
        'recovered' : 'Total Recovered',
        'death' : 'Total Deaths', 'deathIncrease' : 'Deaths Daily',
        'hospitalizedCumulative' : 'Total Hospitalized', 'hospitalizedIncrease' : 'Hospitalized Daily', 'hospitalizedCurrently' : 'Currently Hospitalized',
        'inIcuCumulative' : 'Total In ICU', 'inIcuCurrently' : 'Currently In ICU',
        'onVentilatorCumulative' : 'Total On Ventilator', 'onVentilatorCurrently' : 'Currently On Ventilator'
    }, inplace = True)
states["Time"] = states["Time"].astype(str)
states["Time"] = states["Time"].apply(lambda x: us_date(x))
states["Date"] = pd.to_datetime(states["Time"], format="%m/%d/%Y")
printColumns(states, "Post Rename Columns")
states_input = pd.merge(states, state_codes, how="left", on="State Abbreviation")
merge_order = [
    'Time', 'Date', 'State Abbreviation', 'State Name', 'Country', 'FIPS', 'Status', 'Data Quality', 
    'Total Deaths', 'Deaths Daily', 'Total Recovered',
    'Total Tests', 'Tests Daily', 
    'Total Negative', 'Negative Daily', 'Total Positive', 'Positive Daily',    
    'Total Hospitalized', 'Currently Hospitalized', 'Hospitalized Daily',
    'Total In ICU', 'Currently In ICU',
    'Total On Ventilator', 'Currently On Ventilator', 
    'Downloaded' 
]
states_input = states_input[merge_order]
printColumns(states_input, "States Input Merge Columns")
states_input.head()

us_summary_cols = [
    'Time', 'Date', 'Country',
    'Total Deaths', 'Deaths Daily', 'Total Recovered',
    'Total Tests', 'Tests Daily', 'Total Negative', 'Negative Daily', 'Total Positive', 'Positive Daily',
    'Total Hospitalized', 'Currently Hospitalized', 'Hospitalized Daily', 
    'Total In ICU', 'Currently In ICU', 'Total On Ventilator', 'Currently On Ventilator'
]
us_stats = states_input[us_summary_cols].groupby(['Time','Date','Country']).sum().reset_index()
us_stats["FIPS"] = 0
us_stats["State Abbreviation"] = "US"
us_stats["State Name"] = "United States" 
us_stats["Status"] = 0
us_stats["Downloaded"] = currentTime
us_stats["Data Quality"] = ""
us_stats = us_stats[merge_order]
us_stats.head()

,Time,Date,State Abbreviation,State Name,Country,FIPS,Status,Data Quality,Total Deaths,Deaths Daily,Total Recovered,Total Tests,Tests Daily,Total Negative,Negative Daily,Total Positive,Positive Daily,Total Hospitalized,Currently Hospitalized,Hospitalized Daily,Total In ICU,Currently In ICU,Total On Ventilator,Currently On Ventilator,Downloaded
0,01/22/2020,2020-01-22,US,United States,United States,0,0,,0.0,0,0.0,1.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,2020-10-14 00:39:19.510936
1,01/23/2020,2020-01-23,US,United States,United States,0,0,,0.0,0,0.0,2.0,1,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,2020-10-14 00:39:19.510936
2,01/24/2020,2020-01-24,US,United States,United States,0,0,,0.0,0,0.0,2.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,2020-10-14 00:39:19.510936
3,01/25/2020,2020-01-25,US,United States,United States,0,0,,0.0,0,0.0,2.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,2020-10-14 00:39:19.510936
4,01/26/2020,2020-01-26,US,United States,United States,0,0,,0.0,0,0.0,2.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,2020-10-14 00:39:19.510936


In [12]:
states_input_w_US = pd.concat([states_input, us_stats])
print(states_input_w_US["State Name"].sort_values(ascending = True).unique())
states_input_w_US.head()

['Alabama' 'Alaska' 'American Samoa' 'Arizona' 'Arkansas' 'California'
 'Colorado' 'Commonwealth of the Northern Mariana Islands' 'Connecticut'
 'Delaware' 'District of Columbia' 'Florida' 'Georgia' 'Guam' 'Hawaii'
 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana'
 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi'
 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey'
 'New Mexico' 'New York' 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma'
 'Oregon' 'Pennsylvania' 'Puerto Rico' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'U.S. Virgin Islands' 'United States'
 'Utah' 'Vermont' 'Virginia' 'Washington' 'West Virginia' 'Wisconsin'
 'Wyoming']


,Time,Date,State Abbreviation,State Name,Country,FIPS,Status,Data Quality,Total Deaths,Deaths Daily,Total Recovered,Total Tests,Tests Daily,Total Negative,Negative Daily,Total Positive,Positive Daily,Total Hospitalized,Currently Hospitalized,Hospitalized Daily,Total In ICU,Currently In ICU,Total On Ventilator,Currently On Ventilator,Downloaded
0,10/13/2020,2020-10-13,AK,Alaska,United States,2,0,A,60.0,0,5909.0,514789.0,8862,503750.0,8707,11039.0,155,NaN,59.0,0,NaN,NaN,NaN,8.0,2020-10-14 00:39:19.510936
1,10/13/2020,2020-10-13,AL,Alabama,United States,1,0,A,2665.0,0,71240.0,1221207.0,6597,1074124.0,5737,167193.0,1117,18440.0,823.0,261,1896.0,NaN,1072.0,NaN,2020-10-14 00:39:19.510936
2,10/13/2020,2020-10-13,AR,Arkansas,United States,5,0,A+,1611.0,25,84804.0,1162665.0,7599,1073314.0,7118,94167.0,680,6069.0,601.0,109,NaN,246.0,747.0,105.0,2020-10-14 00:39:19.510936
3,10/13/2020,2020-10-13,AS,American Samoa,United States,60,1,D,0.0,0,NaN,1616.0,0,1616.0,0,0.0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,2020-10-14 00:39:19.510936
4,10/13/2020,2020-10-13,AZ,Arizona,United States,4,0,A+,5767.0,8,37403.0,1581151.0,6951,1359347.0,6286,226734.0,684,20312.0,706.0,55,NaN,146.0,NaN,88.0,2020-10-14 00:39:19.510936


In [13]:
printColumns(us_state_demographics, "State Demographics Columns")
states_input_w_demographics = pd.merge(states_input_w_US, us_state_demographics, how="left", on="State Abbreviation")
printColumns(states_input_w_demographics,"State Input Columns")
states_input_order = [
    'Time', 'Date', 'FIPS', 'State Abbreviation', 'State Name', 'Status', 'Census Region', 'Country',  'Data Quality',
    'Total Tests', 'Tests Daily', 'Total Negative', 'Negative Daily', 
    'Total Positive', 'Positive Daily', 
    'Total Deaths', 'Deaths Daily', 'Total Recovered',
    'Total Hospitalized', 'Currently Hospitalized', 'Hospitalized Daily', 
    'Total In ICU', 'Currently In ICU', 'Total On Ventilator', 'Currently On Ventilator', 
    'Population 2019', 'Female', 'Male', 'Pct Male', 'Pct Female',
    '< 1', 1, 2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
    40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
    70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, '85+',
    '1-4', '5-14', '15-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84',
    'Pct < 1', 'Pct 1-4', 'Pct 5-14', 'Pct 15-24', 'Pct 25-34', 'Pct 35-44', 'Pct 45-54', 'Pct 55-64', 'Pct 65-74', 'Pct 75-84', 'Pct 85+'
]
states_input_w_demographics = states_input_w_demographics[states_input_order]
print(states_input_w_demographics["Census Region"].sort_values(ascending = True).unique())
states_input_w_demographics["Census Region"] = states_input_w_demographics["Census Region"].astype(str)
regions = ['Midwest','Northeast','South','West']
states_input_w_demographics["Census Region"] = states_input_w_demographics["Census Region"].apply(lambda x: x if x in regions else "Other")
states_input_w_demographics = states_input_w_demographics.sort_values(["FIPS"])
states_input_w_demographics.head()

State Demographics Columns
Index(['State Abbreviation',      'Census Region',    'Population 2019',
                   'Female',               'Male',           'Pct Male',
               'Pct Female',                '< 1',                    1,
                          2,
       ...
                  'Pct 1-4',           'Pct 5-14',          'Pct 15-24',
                'Pct 25-34',          'Pct 35-44',          'Pct 45-54',
                'Pct 55-64',          'Pct 65-74',          'Pct 75-84',
                  'Pct 85+'],
      dtype='object', length=113)
State Input Columns
Index([              'Time',               'Date', 'State Abbreviation',
               'State Name',            'Country',               'FIPS',
                   'Status',       'Data Quality',       'Total Deaths',
             'Deaths Daily',
       ...
                  'Pct 1-4',           'Pct 5-14',          'Pct 15-24',
                'Pct 25-34',          'Pct 35-44',          'Pct 45-54',
      

,Time,Date,FIPS,State Abbreviation,State Name,Status,Census Region,Country,Data Quality,Total Tests,Tests Daily,Total Negative,Negative Daily,Total Positive,Positive Daily,Total Deaths,Deaths Daily,Total Recovered,Total Hospitalized,Currently Hospitalized,Hospitalized Daily,Total In ICU,Currently In ICU,Total On Ventilator,Currently On Ventilator,Population 2019,Female,Male,Pct Male,Pct Female,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
12798,10/13/2020,2020-10-13,0,US,United States,0,Other,United States,,117357626.0,894411,103943238.0,793495,7817857.0,46647,207557.0,669,3124593.0,426255.0,36034.0,1960,21735.0,7020.0,2485.0,1756.0,327052602.0,166382969.0,160669633.0,0.491265,0.508735,3783052.0,3829599.0,3922044.0,3998665.0,4043323.0,4028281.0,4017227.0,4022319.0,4066194.0,4061874.0,4060940.0,4189261.0,4208387.0,4175221.0,4164459.0,4175459.0,4150420.0,4141202.0,4222496.0,4272909.0,4203852.0,4207331.0,4225571.0,4271168.0,4379797.0,4478029.0,4555029.0,4680384.0,4770188.0,4761228.0,4572098.0,4461418.0,4382364.0,4396137.0,4425400.0,4282329.0,4340779.0,4330751.0,4278041.0,4358571.0,4084430.0,4002096.0,3958280.0,3840006.0,3954899.0,3827009.0,3879028.0,4049147.0,4275506.0,4324119.0,4091962.0,4000602.0,3998899.0,4066500.0,4303847.0,4373250.0,4360049.0,4341663.0,4385025.0,4413491.0,4252458.0,4215037.0,4156618.0,3996074.0,3950547.0,3774597.0,3618069.0,3464437.0,3345475.0,3252423.0,3136704.0,3083083.0,3191048.0,2334433.0,2283164.0,2198286.0,2222392.0,1911261.0,1720817.0,1599909.0,1475278.0,1381641.0,1241341.0,1151190.0,1067757.0,6604958.0,15793631.0,40994163.0,42250205.0,45482275.0,41430182.0,40816619.0,42444212.0,31483433.0,15969872.0,0.011567,0.048291,0.125344,0.129185,0.139067,0.126677,0.124801,0.129778,0.096264,0.04883,0.020195
12629,04/27/2020,2020-04-27,0,US,United States,0,Other,United States,,5749371.0,201690,4658012.0,173380,993356.0,22708,52653.0,1270,121323.0,116002.0,56189.0,3076,3720.0,13812.0,195.0,4867.0,327052602.0,166382969.0,160669633.0,0.491265,0.508735,3783052.0,3829599.0,3922044.0,3998665.0,4043323.0,4028281.0,4017227.0,4022319.0,4066194.0,4061874.0,4060940.0,4189261.0,4208387.0,4175221.0,4164459.0,4175459.0,4150420.0,4141202.0,4222496.0,4272909.0,4203852.0,4207331.0,4225571.0,4271168.0,4379797.0,4478029.0,4555029.0,4680384.0,4770188.0,4761228.0,4572098.0,4461418.0,4382364.0,4396137.0,4425400.0,4282329.0,4340779.0,4330751.0,4278041.0,4358571.0,4084430.0,4002096.0,3958280.0,3840006.0,3954899.0,3827009.0,3879028.0,4049147.0,4275506.0,4324119.0,4091962.0,4000602.0,3998899.0,4066500.0,4303847.0,4373250.0,4360049.0,4341663.0,4385025.0,4413491.0,4252458.0,4215037.0,4156618.0,3996074.0,3950547.0,3774597.0,3618069.0,3464437.0,3345475.0,3252423.0,3136704.0,3083083.0,3191048.0,2334433.0,2283164.0,2198286.0,2222392.0,1911261.0,1720817.0,1599909.0,1475278.0,1381641.0,1241341.0,1151190.0,1067757.0,6604958.0,15793631.0,40994163.0,42250205.0,45482275.0,41430182.0,40816619.0,42444212.0,31483433.0,15969872.0,0.011567,0.048291,0.125344,0.129185,0.139067,0.126677,0.124801,0.129778,0.096264,0.04883,0.020195
12628,04/26/2020,2020-04-26,0,US,United States,0,Other,United States,,5547681.0,204025,4484632.0,179284,970648.0,27310,51383.0,1215,116529.0,112926.0,56192.0,2192,2571.0,14104.0,227.0,5121.0,327052602.0,166382969.0,160669633.0,0.491265,0.508735,3783052.0,3829599.0,3922044.0,3998665.0,4043323.0,4028281.0,4017227.0,4022319.0,4066194.0,4061874.0,4060940.0,4189261.0,4208387.0,4175221.0,4164459.0,4175459.0,4150420.0,4141202.0,4222496.0,4272909.0,4203852.0,4207331.0,4225571.0,4271168.0,4379797.0,4478029.0,4555029.0,4680384.0,4770188.0,4761228.0,4572098.0,4461418.0,4382364.0,4396137.0,44

In [15]:
states_input_integers = [
    'Total Deaths', 'Deaths Daily',
    'Total Hospitalized', 'Currently Hospitalized', 'Hospitalized Daily',
    'Total In ICU', 'Currently In ICU',
    'Total Negative', 'Negative Daily',
    'Total On Ventilator', 'Currently On Ventilator',
    'Total Positive', 'Positive Daily',
    'Total Recovered',
    'Total Tests', 'Tests Daily', 
    'FIPS', 'Status',
    'Female', 'Male', 'Population 2019',
    '< 1', 1, 2, 3, 4, 5, 6, 7, 8, 9,
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
    30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
    40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
    50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
    60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
    70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, '85+',
    '1-4', '5-14', '15-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84'
]
states_input_float = [
    'Pct Male', 'Pct Female',
    'Pct < 1', 'Pct 1-4', 'Pct 5-14', 'Pct 15-24', 'Pct 25-34', 'Pct 35-44', 'Pct 45-54', 'Pct 55-64', 'Pct 65-74', 'Pct 75-84', 'Pct 85+'
]
states_input_cleaned = states_input_w_demographics.copy()
printColumns(states_input_cleaned, "States Input Columns")
for i in states_input_cleaned.columns:
    print(i)
    if (i in states_input_integers):
        states_input_cleaned[i] = states_input_cleaned[i].apply(lambda x: pd.to_numeric(x,
                                                                                        errors='coerce',
                                                                                        downcast='integer'))
        states_input_cleaned[i] = states_input_cleaned[i].astype(str)
        states_input_cleaned[i] = states_input_cleaned[i].apply(lambda x: removeDecimal(x))
        states_input_cleaned[i] = states_input_cleaned[i].apply(lambda x: pd.to_numeric(x,
                                                                                        errors='coerce',
                                                                                        downcast='integer'))
    else:
        states_input_cleaned[i] = states_input_cleaned[i].astype(str)
    if (i in states_input_float):
        states_input_cleaned[i] = states_input_cleaned[i].apply(lambda x: pd.to_numeric(x,
                                                                                        errors='coerce'))
    states_input_cleaned[i] = states_input_cleaned[i].astype(str)
    states_input_cleaned[i] = states_input_cleaned[i].apply(lambda x: emptyNan(x))
states_input_cleaned = states_input_cleaned[states_input_order]
states_input_cleaned = states_input_cleaned.sort_values(["FIPS"])
states_input_cleaned.head()

States Input Columns
Index([              'Time',               'Date',               'FIPS',
       'State Abbreviation',         'State Name',             'Status',
            'Census Region',            'Country',       'Data Quality',
              'Total Tests',
       ...
                  'Pct 1-4',           'Pct 5-14',          'Pct 15-24',
                'Pct 25-34',          'Pct 35-44',          'Pct 45-54',
                'Pct 55-64',          'Pct 65-74',          'Pct 75-84',
                  'Pct 85+'],
      dtype='object', length=136)
Time
Date
FIPS
State Abbreviation
State Name
Status
Census Region
Country
Data Quality
Total Tests
Tests Daily
Total Negative
Negative Daily
Total Positive
Positive Daily
Total Deaths
Deaths Daily
Total Recovered
Total Hospitalized
Currently Hospitalized
Hospitalized Daily
Total In ICU
Currently In ICU
Total On Ventilator
Currently On Ventilator
Population 2019
Female
Male
Pct Male
Pct Female
< 1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1

,Time,Date,FIPS,State Abbreviation,State Name,Status,Census Region,Country,Data Quality,Total Tests,Tests Daily,Total Negative,Negative Daily,Total Positive,Positive Daily,Total Deaths,Deaths Daily,Total Recovered,Total Hospitalized,Currently Hospitalized,Hospitalized Daily,Total In ICU,Currently In ICU,Total On Ventilator,Currently On Ventilator,Population 2019,Female,Male,Pct Male,Pct Female,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
12798,10/13/2020,2020-10-13,0,US,United States,0,Other,United States,,117357626.0,894411,103943238.0,793495,7817857.0,46647,207557.0,669,3124593.0,426255.0,36034.0,1960,21735.0,7020.0,2485.0,1756.0,327052602.0,166382969.0,160669633.0,0.4912654172982241,0.5087345827017759,3783052.0,3829599.0,3922044.0,3998665.0,4043323.0,4028281.0,4017227.0,4022319.0,4066194.0,4061874.0,4060940.0,4189261.0,4208387.0,4175221.0,4164459.0,4175459.0,4150420.0,4141202.0,4222496.0,4272909.0,4203852.0,4207331.0,4225571.0,4271168.0,4379797.0,4478029.0,4555029.0,4680384.0,4770188.0,4761228.0,4572098.0,4461418.0,4382364.0,4396137.0,4425400.0,4282329.0,4340779.0,4330751.0,4278041.0,4358571.0,4084430.0,4002096.0,3958280.0,3840006.0,3954899.0,3827009.0,3879028.0,4049147.0,4275506.0,4324119.0,4091962.0,4000602.0,3998899.0,4066500.0,4303847.0,4373250.0,4360049.0,4341663.0,4385025.0,4413491.0,4252458.0,4215037.0,4156618.0,3996074.0,3950547.0,3774597.0,3618069.0,3464437.0,3345475.0,3252423.0,3136704.0,3083083.0,3191048.0,2334433.0,2283164.0,2198286.0,2222392.0,1911261.0,1720817.0,1599909.0,1475278.0,1381641.0,1241341.0,1151190.0,1067757.0,6604958.0,15793631.0,40994163.0,42250205.0,45482275.0,41430182.0,40816619.0,42444212.0,31483433.0,15969872.0,0.011567105648650364,0.048290797576348285,0.12534424966904864,0.1291847389124273,0.13906715531955927,0.12667742664832854,0.12480138898268114,0.12977793706713886,0.0962641263438106,0.04882967419412245,0.02019539963788455
12666,06/03/2020,2020-06-03,0,US,United States,0,Other,United States,,18704512.0,470848,16545978.0,448319,1855946.0,20390,103118.0,993,557366.0,217100.0,33070.0,2255,8688.0,7227.0,717.0,3749.0,327052602.0,166382969.0,160669633.0,0.4912654172982241,0.5087345827017759,3783052.0,3829599.0,3922044.0,3998665.0,4043323.0,4028281.0,4017227.0,4022319.0,4066194.0,4061874.0,4060940.0,4189261.0,4208387.0,4175221.0,4164459.0,4175459.0,4150420.0,4141202.0,4222496.0,4272909.0,4203852.0,4207331.0,4225571.0,4271168.0,4379797.0,4478029.0,4555029.0,4680384.0,4770188.0,4761228.0,4572098.0,4461418.0,4382364.0,4396137.0,4425400.0,4282329.0,4340779.0,4330751.0,4278041.0,4358571.0,4084430.0,4002096.0,3958280.0,3840006.0,3954899.0,3827009.0,3879028.0,4049147.0,4275506.0,4324119.0,4091962.0,4000602.0,3998899.0,4066500.0,4303847.0,4373250.0,4360049.0,4341663.0,4385025.0,4413491.0,4252458.0,4215037.0,4156618.0,3996074.0,3950547.0,3774597.0,3618069.0,3464437.0,3345475.0,3252423.0,3136704.0,3083083.0,3191048.0,2334433.0,2283164.0,2198286.0,2222392.0,1911261.0,1720817.0,1599909.0,1475278.0,1381641.0,1241341.0,1151190.0,1067757.0,6604958.0,15793631.0,40994163.0,42250205.0,45482275.0,41430182.0,40816619.0,42444212.0,31483433.0,15969872.0,0.011567105648650364,0.048290797576348285,0.12534424966904864,0.1291847389124273,0.13906715531955927,0.12667742664832854,0.12480138898268114,0.12977793706713886,0.0962641263438106,0.04882967419412245,0.02019539963788455
12796,10/11/2020,2020-10-11,0,US,United States,0,Other,United States,,115456293.0,957345,102260309.0,803858,7728086.0,46946,206601.0,466,3075110.0,423106.0,34490.0,1015,21553.0,6679.0,2454.0,1646.0,327052602.0,166382969.0,160669633.0,0.4912654172982241,0.5087345827017759,3783052.0,3829599.0,3922044.0

In [16]:
states_input_cleaned.to_excel(r'C:\Users\janin\Downloads\states_input.xlsx', index = False)

US Statistics

In [ ]:
pgmm_us = pd.read_excel(r'C:\Users\janin\Downloads\USState-Results.xlsx')

Download Raw Country Data

In [14]:
def print_column_unique(column):
    print("Values:")
    values = column.sort_values(ascending = True).unique()
    print(values)
    return values

def print_column_missing(column, comparison):
    values = print_column_unique(column)
    print("Comparison:")
    print(comparison)
    missing_values = []
    for value in values:
        if not value in comparison:
            missing_values.append(value)
    if len(missing_values) > 0:
        print("Missing Values:")
        print(missing_values)
    else:
        print("No missing values")
    return values

In [15]:
# Remove special characters and rename Congos
conversions = {}
def fixCountry(country):
    converted_country = ""
    if country == "Congo - Kinshasa":
        converted_country =  "Democratic Republic of Congo"
    elif country == "Congo - Brazzaville":
        converted_country =  "Republic of the Congo"
    elif country == "Sao Tome and Principe" or country == "Sao Tome & Príncipe" or country == "São Tomé & Príncipe":
        converted_country = "São Tomé and Príncipe"
    elif country == "Cote d'Ivoire" or country == "Cote dIvoire":
        converted_country = "Côte d’Ivoire"
    elif country == "Eswatini":
        converted_country = "Swaziland"
    elif country == "Cape Verde":
        converted_country = "Cabo Verde"
    else:
        converted_country = country
    if converted_country != country:
        if converted_country not in conversions:
            conversions[country] = 1
        else:
            conversions[country] = conversions[country] + 1
    return converted_country.strip()

def fixCountries(countries_column, configuredCountries):
    conversions = {}
    countries_conversion = countries_column.astype(str)
    countries_conversion = countries_conversion.apply(lambda x: fixCountry(x))
    print(conversions)
    countries = print_column_missing(countries_conversion,configuredCountries)
    return countries_conversion

def region(country):
    if country in sub_saharan_african_countries:
        return "Sub-Saharan Africa"
    elif country in south_asia_countries:
        return "South Asia"
    elif country in latin_american_countries:
        return "Latin America"
    else:
        return ""